In [3]:
import csv
import pandas as pd
import polars as pl
import re
import io
import os
os.chdir('/home/matebarey/Downloads/repos/mozity')

In [ ]:
trim = lambda s: s.strip()
upper = lambda s: s.upper()
exclaim = lambda s: s + "!"


In [10]:
exclaim(upper(trim(" get off my lawn")))

'GET OFF MY LAWN!'

### Function composition

A composition of these functions is useful for combining the effects of these functions. Given two functions, $f$ and $g$,

$$
\begin{align}
f : A & \rightarrow B \\
g : B & \rightarrow C \\
\end{align}
$$

the composition of $f$ and $g$ is defined as

$$
\begin{align}
g \circ f: A & \rightarrow C \\
\end{align}
$$

where

$$\text{compose(f, g)(x)} = (g\circ f) (x) = g(f(x))$$

In [12]:
#function composition
compose = lambda f, g: lambda x: g(f(x))

enthuse = compose(compose(trim,upper),exclaim)
enthuse(" get off my lawn")

'GET OFF MY LAWN!'

Composition is associative. Given three functions

$$
\begin{align}
f : A & \rightarrow B \\
g : B & \rightarrow C \\
h : C & \rightarrow D \\
\end{align}
$$

then 

$$h\circ g\circ f = (h\circ g)\circ f = h\circ (g\circ f)$$

The functions `map`, `filter`, and `functools.reduce` extend the utility of pure functions. Note that `map` actually returns an iterator, so `list` is required to apply the function to every element of the data given in `map`.

In [13]:
list(map(enthuse, ["hello", "go away", " get off my lawn"]))

['HELLO!', 'GO AWAY!', 'GET OFF MY LAWN!']

`reduce` applies a function of two values recursively to a tuple, list, or iterator to reduce it to a single cumulative value. This provides a convenient means to construct of composition of two or more functions.

In [14]:
from functools import reduce

enthuse = reduce(compose, (trim, upper, exclaim))
enthuse(" get off my lawn")

'GET OFF MY LAWN!'

This may not be a desirable result for a more complex workflow. This is where monads fit in.

A monad wraps the inputs and outputs of a pure function into a structure that can perform additional computation. There two basic methods for monads:

* **unit** which wraps a value into a monad.
* **bind** which chains pure functions together.

### The **Maybe** monad

 The following cell shows a Python implementation for a **Maybe** monad. The **Maybe** monad wraps a value the may contain a value. If the result of bind produces an error, the value stored is `None` and a flag is set.

In [ ]:
# phase 1 load and understand

#1) detact and strip byte order Mark
#special unicode char
#2) parse using csv
#quotechar='""
import re
from pathlib import Path
from functools import reduce

'''
old school notes
sais cleanup
def clean_data(obj):
    return ((re.sub(r'\W+', ' ', obj)).lower()).strip()



#georges name cleanup for parsing was
    def str_enc_correct(input_str):
        input_str = input_str.replace(u'\u201c', '"').replace(u'\u201d', '"').replace(u'\xa0', u' ')

        string_encode = input_str.encode('ascii','ignore')
        string_decode = string_encode.decode('ascii')
        #str_f = re.sub('[^a-zA-Z0-9-_*.&\/()\\~,;$#\"]', ' ', string_decode)

        #`, !, @,^ ,  [, ], {, },  <, >, ?
        str_f = re.sub('[^a-zA-Z0-9-_*.&\/()\\~,;$#\":\-+%=`!@\^\[\]\{\}\<\>\?]', ' ', string_decode)

        return str_f
'''

def csv_info(path, has_header=True):
    """
    Analyze and clean CSV bytes for any parser.
    """
    b = Path(path).read_bytes()

    # Helper lambdas
    count_pattern = lambda p, data: len(re.findall(p, data))
    is_regex = lambda p: isinstance(p, bytes) and (b'(?!' in p or b'(?<!' in p or b'^' in p)
    apply_fix = lambda data, fix: re.sub(fix[0], fix[1], data) if is_regex(fix[0]) else data.replace(fix[0], fix[1])

    # What to count
    patterns = {
        'crlf': b'\r\n',
        'lf': rb'(?<!\r)\n',
        'cr': rb'\r(?!\n)',
        'bom_start': rb'^\xef\xbb\xbf',
        'bom_interior': rb'(?<!^)\xef\xbb\xbf',
        'null': b'\x00',
        'sep_header': rb'(?i)^sep=.*?\n',
        'comma': b',',
        'tab': b'\t',
        'semicolon': b';',
        'pipe': b'|'
    }

    # Count everything
    counts = {k: count_pattern(p, b) for k, p in patterns.items()}

        # How to fix issues
    fixes = {
        'crlf': (b'\r\n', b'\n'),
        'cr': (rb'\r(?!\n)', b'\n'),
        'bom_start': (rb'^\xef\xbb\xbf', b''),
        'bom_interior': (b'\xef\xbb\xbf', b''),
        'null': (b'\x00', b''),
        'sep_header': (rb'(?i)^sep=.*?\n', b''),

        # Quote normalization - convert all quote variants to standard quotes
        'curly_quotes_open': (b'\xe2\x80\x9c', b'"'),      # " (left double quotation mark)
        'curly_quotes_close': (b'\xe2\x80\x9d', b'"'),     # " (right double quotation mark)
        'curly_single_open': (b'\xe2\x80\x98', b"'"),      # ' (left single quotation mark)
        'curly_single_close': (b'\xe2\x80\x99', b"'"),     # ' (right single quotation mark)
        'grave_accent': (b'\x60', b"'"),                   # ` (grave accent used as quote)
        'acute_accent': (b'\xc2\xb4', b"'"),               # ´ (acute accent used as quote)

        # After normalizing, fix unescaped quotes
        'unescaped_quotes': (rb'(?<!^)(?<!,)"(?!,)(?!$)(?!")', b'""')
    }

    # Apply only needed fixes
    needed_fixes = [(p, r) for k, (p, r) in fixes.items() if counts[k] > 0]
    cleaned = reduce(apply_fix, needed_fixes, b)

    # Find delimiter
    delim_map = {'comma': ',', 'tab': '\t', 'semicolon': ';', 'pipe': '|'}
    delimiter = max(delim_map, key=lambda k: counts[k])

    return {
        'cleaned': cleaned,
        'delimiter': delim_map[delimiter],
        'issues': [k for k, v in counts.items() if v > 0 and k in fixes],
        'has_header': has_header
    }
csv_exam_1 = 'snippets.csv'
csv_exam_1i = csv_info(csv_exam_1)
cleaned = csv_exam_1i['cleaned']
delimiter = csv_exam_1i['delimiter']
df = pl.read_csv(cleaned,separator=delimiter)
df

ComputeError: could not parse `""Quoted line"", with ""quotes"".",2025-05-22T09:09:09Z
7,"Polish diacritics","i18n","search","Wyszukaj słowo „żółć” i jego warianty bez znaków diakrytycznych.",2025-06-01T11:11:11Z
` as dtype `str` at column 'id,title,category,tags,body,created_at' (column number 1)

The current offset in the file is 833 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `""Quoted line"", with ""quotes"".",2025-05-22T09:09:09Z
7,"Polish diacritics","i18n","search","Wyszukaj słowo „żółć” i jego warianty bez znaków diakrytycznych.",2025-06-01T11:11:11Z
` to the `null_values` list.

Original error: ```invalid csv file

Field `""Quoted line"", with ""quotes"".",2025-05-22T09:09:09Z
7,"Polish diacritics","i18n","search","Wyszukaj słowo „żółć” i jego warianty bez znaków diakrytycznych.",2025-06-01T11:11:11Z
` is not pr